### 핵심 동작 원리

첫 번째 카드: 항상 랜덤 선택
기억 검색: 첫 번째 카드와 같은 색상을 기억에서 찾기
두 번째 카드 선택:

기억에 있으면 → 그 위치로 직접 이동 <br>
기억에 없으면 → 랜덤 선택


메모리 관리: FIFO 방식으로 오래된 것부터 제거
매칭 성공시: 해당 카드들을 기억에서 제거

In [9]:
import random
import numpy as np
from collections import deque
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt

class Card:
    """카드 클래스"""
    def __init__(self, color: int, position: int):
        self.color = color
        self.position = position
        self.is_matched = False

class LimitedMemoryPlayer:
    """최근 N개 카드만 기억하는 플레이어 클래스"""
    def __init__(self, memory_size: int = 3):
        self.memory_size = memory_size
        self.memory = {}  # position -> color 매핑
        self.memory_order = deque(maxlen=memory_size)  # FIFO 순서 관리
        
    def remember_card(self, position: int, color: int):
        """카드를 기억에 추가"""
        # 이미 기억하고 있는 위치라면 순서만 업데이트
        if position in self.memory:
            self.memory_order.remove(position)
            self.memory_order.append(position)
        else:
            # 메모리가 가득 찼다면 가장 오래된 것 제거
            if len(self.memory) >= self.memory_size:
                oldest_pos = self.memory_order.popleft()
                del self.memory[oldest_pos]
            
            self.memory[position] = color
            self.memory_order.append(position)
    
    def find_matching_card(self, target_color: int, exclude_positions: List[int] = None) -> int:
        """기억에서 같은 색상의 카드 위치 찾기"""
        if exclude_positions is None:
            exclude_positions = []
            
        for position, color in self.memory.items():
            if color == target_color and position not in exclude_positions:
                return position
        return -1
    
    def remove_from_memory(self, positions: List[int]):
        """매칭된 카드들을 기억에서 제거"""
        for pos in positions:
            if pos in self.memory:
                del self.memory[pos]
                if pos in self.memory_order:
                    self.memory_order.remove(pos)
    
    def get_memory_info(self):
        """디버깅용 메모리 상태 반환"""
        return dict(self.memory)

def create_deck(n_pairs: int) -> List[Card]:
    """n개 쌍의 카드 덱 생성"""
    deck = []
    colors = list(range(n_pairs)) * 2  # 각 색상마다 2장
    random.shuffle(colors)
    
    for i, color in enumerate(colors):
        deck.append(Card(color, i))
    
    return deck

def get_available_positions(deck: List[Card]) -> List[int]:
    """아직 매칭되지 않은 카드 위치들 반환"""
    return [i for i, card in enumerate(deck) if not card.is_matched]

def simulate_game_with_limited_memory(n_pairs: int, memory_size: int = 3, debug: bool = False) -> Tuple[int, bool, Dict]:
    """제한된 기억력으로 게임 시뮬레이션"""
    deck = create_deck(n_pairs)
    player = LimitedMemoryPlayer(memory_size)
    turns = 0
    matched_pairs = 0
    total_flips = 0
    debug_info = {"turns_detail": [], "memory_states": []}
    
    while matched_pairs < n_pairs:
        turns += 1
        turn_info = {"turn": turns, "before_memory": player.get_memory_info()}
        
        # 사용 가능한 카드들
        available_positions = get_available_positions(deck)
        
        if len(available_positions) < 2:
            break
        
        # 첫 번째 카드 선택 (랜덤)
        first_pos = random.choice(available_positions)
        first_card = deck[first_pos]
        total_flips += 1
        
        # 첫 번째 카드를 기억에 추가
        player.remember_card(first_pos, first_card.color)
        
        turn_info["first_card"] = {"pos": first_pos, "color": first_card.color}
        
        # 기억에서 같은 색상의 카드 찾기
        matching_pos = player.find_matching_card(first_card.color, [first_pos])
        
        if matching_pos != -1 and matching_pos in available_positions:
            # 기억에서 매칭 카드를 찾은 경우
            second_pos = matching_pos
            second_card = deck[second_pos]
            total_flips += 1
            
            turn_info["second_card"] = {"pos": second_pos, "color": second_card.color, "from_memory": True}
            
            # 매칭 성공 (기억이 정확했으므로)
            first_card.is_matched = True
            second_card.is_matched = True
            matched_pairs += 1
            
            # 매칭된 카드들을 기억에서 제거
            player.remove_from_memory([first_pos, second_pos])
            
            turn_info["result"] = "match"
        else:
            # 기억에 없으므로 랜덤하게 두 번째 카드 선택
            available_for_second = [pos for pos in available_positions if pos != first_pos]
            
            if not available_for_second:
                break
                
            second_pos = random.choice(available_for_second)
            second_card = deck[second_pos]
            total_flips += 1
            
            turn_info["second_card"] = {"pos": second_pos, "color": second_card.color, "from_memory": False}
            
            # 두 번째 카드를 기억에 추가
            player.remember_card(second_pos, second_card.color)
            
            if first_card.color == second_card.color:
                # 운 좋게 매칭
                first_card.is_matched = True
                second_card.is_matched = True
                matched_pairs += 1
                
                # 매칭된 카드들을 기억에서 제거
                player.remove_from_memory([first_pos, second_pos])
                
                turn_info["result"] = "lucky_match"
            else:
                turn_info["result"] = "no_match"
        
        turn_info["after_memory"] = player.get_memory_info()
        turn_info["matched_pairs"] = matched_pairs
        
        if debug:
            debug_info["turns_detail"].append(turn_info)
            debug_info["memory_states"].append(dict(player.memory))
    
    success = matched_pairs == n_pairs
    debug_info["final_stats"] = {
        "turns": turns,
        "total_flips": total_flips,
        "matched_pairs": matched_pairs,
        "success": success
    }
    
    return turns, success, debug_info

def run_simulation_batch(n_pairs: int, memory_size: int, num_simulations: int) -> Dict:
    """배치 시뮬레이션 실행"""
    results = {
        'turns': [],
        'success_rate': 0,
        'avg_turns': 0,
        'std_turns': 0,
        'min_turns': float('inf'),
        'max_turns': 0
    }
    
    successful_games = 0
    
    for i in range(num_simulations):
        turns, success, debug_info = simulate_game_with_limited_memory(n_pairs, memory_size)
        results['turns'].append(turns)
        
        if success:
            successful_games += 1
            
        results['min_turns'] = min(results['min_turns'], turns)
        results['max_turns'] = max(results['max_turns'], turns)
        
        # 처음 몇 개는 디버그 정보 출력
        if i < 3 and n_pairs <= 5:
            print(f"  시뮬레이션 {i+1}: {turns}턴, 성공: {success}")
    
    results['success_rate'] = successful_games / num_simulations
    results['avg_turns'] = np.mean(results['turns'])
    results['std_turns'] = np.std(results['turns'])
    
    return results

def compare_memory_sizes(n_pairs: int, memory_sizes: List[int], num_simulations: int = 10000):
    """다양한 기억 크기 비교"""
    comparison_results = {}
    
    print(f"카드 쌍 수: {n_pairs} (총 {2*n_pairs}장)")
    print("="*70)
    
    for memory_size in memory_sizes:
        print(f"기억 크기 {memory_size}개 시뮬레이션 중...")
        results = run_simulation_batch(n_pairs, memory_size, num_simulations)
        comparison_results[memory_size] = results
        
        print(f"기억 크기: {memory_size}개")
        print(f"  평균 턴 수: {results['avg_turns']:.2f} ± {results['std_turns']:.2f}")
        print(f"  범위: {results['min_turns']} ~ {results['max_turns']} 턴")
        print(f"  성공률: {results['success_rate']:.1%}")
        print()
    
    return comparison_results

def debug_single_game(n_pairs: int, memory_size: int):
    """단일 게임 디버그 실행"""
    print(f"=== 디버그: {n_pairs}쌍, 기억크기 {memory_size} ===")
    
    turns, success, debug_info = simulate_game_with_limited_memory(
        n_pairs, memory_size, debug=True
    )
    
    print(f"결과: {turns}턴, 성공: {success}")
    print("\n턴별 상세:")
    
    for turn_detail in debug_info["turns_detail"]:
        print(f"턴 {turn_detail['turn']}:")
        print(f"  첫 번째: 위치{turn_detail['first_card']['pos']} 색상{turn_detail['first_card']['color']}")
        print(f"  두 번째: 위치{turn_detail['second_card']['pos']} 색상{turn_detail['second_card']['color']} "
              f"({'기억에서' if turn_detail['second_card']['from_memory'] else '랜덤'})")
        print(f"  결과: {turn_detail['result']}")
        print(f"  매칭된 쌍: {turn_detail['matched_pairs']}/{n_pairs}")
        print(f"  기억 상태: {turn_detail['after_memory']}")
        print()

# 실행 예제
if __name__ == "__main__":
    # 1. 먼저 작은 규모로 디버그
    print("=== 디버그 테스트 ===")
    debug_single_game(2, 1)  # 2쌍, 기억크기 1
    print()
    debug_single_game(2, 3)  # 2쌍, 기억크기 3
    print()
    
    # 2. 기본 시뮬레이션
    print("=== 카드 뒤집기 게임 시뮬레이션 (제한된 기억력) ===\n")
    
    n_pairs = 10
    memory_sizes = [1, 2, 3, 4, 5, 8]
    
    comparison_results = compare_memory_sizes(n_pairs, memory_sizes, 5000)
    
    # 3. 이론적 완벽 플레이어와 비교
    theoretical_perfect = 1.61 * n_pairs
    print(f"이론적 완벽 플레이어 기대 턴 수: {theoretical_perfect:.2f}")
    if 8 in comparison_results:
        print(f"기억 크기 8개 플레이어 평균 턴 수: {comparison_results[8]['avg_turns']:.2f}")
        print(f"차이: {comparison_results[8]['avg_turns'] - theoretical_perfect:.2f} 턴")
    
    # 4. 기억 크기별 효율성 분석
    print("\n=== 기억 크기별 효율성 분석 ===")
    baseline_turns = comparison_results[1]['avg_turns']  # 기억 크기 1개 기준
    
    for memory_size in memory_sizes:
        if memory_size in comparison_results:
            turns = comparison_results[memory_size]['avg_turns']
            improvement = (baseline_turns - turns) / baseline_turns * 100
            print(f"기억 크기 {memory_size}개: {improvement:+.1f}% 개선 "
                  f"({turns:.2f}턴 vs {baseline_turns:.2f}턴)")

=== 디버그 테스트 ===
=== 디버그: 2쌍, 기억크기 1 ===
결과: 5턴, 성공: True

턴별 상세:
턴 1:
  첫 번째: 위치3 색상1
  두 번째: 위치1 색상0 (랜덤)
  결과: no_match
  매칭된 쌍: 0/2
  기억 상태: {1: 0}

턴 2:
  첫 번째: 위치1 색상0
  두 번째: 위치0 색상1 (랜덤)
  결과: no_match
  매칭된 쌍: 0/2
  기억 상태: {0: 1}

턴 3:
  첫 번째: 위치2 색상0
  두 번째: 위치3 색상1 (랜덤)
  결과: no_match
  매칭된 쌍: 0/2
  기억 상태: {3: 1}

턴 4:
  첫 번째: 위치2 색상0
  두 번째: 위치1 색상0 (랜덤)
  결과: lucky_match
  매칭된 쌍: 1/2
  기억 상태: {}

턴 5:
  첫 번째: 위치3 색상1
  두 번째: 위치0 색상1 (랜덤)
  결과: lucky_match
  매칭된 쌍: 2/2
  기억 상태: {}


=== 디버그: 2쌍, 기억크기 3 ===
결과: 2턴, 성공: True

턴별 상세:
턴 1:
  첫 번째: 위치0 색상0
  두 번째: 위치1 색상0 (랜덤)
  결과: lucky_match
  매칭된 쌍: 1/2
  기억 상태: {}

턴 2:
  첫 번째: 위치3 색상1
  두 번째: 위치2 색상1 (랜덤)
  결과: lucky_match
  매칭된 쌍: 2/2
  기억 상태: {}


=== 카드 뒤집기 게임 시뮬레이션 (제한된 기억력) ===

카드 쌍 수: 10 (총 20장)
기억 크기 1개 시뮬레이션 중...
기억 크기: 1개
  평균 턴 수: 100.08 ± 35.34
  범위: 21 ~ 291 턴
  성공률: 100.0%

기억 크기 2개 시뮬레이션 중...
기억 크기: 2개
  평균 턴 수: 59.32 ± 18.33
  범위: 20 ~ 163 턴
  성공률: 100.0%

기억 크기 3개 시뮬레이션 중...
기억 크기: 3개
  평균 턴 수: 40.76 ± 11.8

## 현재 게임 규칙

1단계2x2 / 총 그림 2개- 기본 카드: 2개- 트릭 카드: 0개제한 시간: 5초성공 리워드: 2원실패 리워드: 1원

2단계2x3 / 총 그림 3개- 기본 카드: 3개- 트릭 카드: 0개제한 시간: 7초성공 리워드: 2원

3단계3x4 / 총 그림 6개- 기본 카드: 6개- 트릭 카드: 0개제한 시간: 9초성공 리워드: 2원

4단계4x4 / 총 그림 8개- 기본 카드: 8개- 트릭 카드: 0개제한 시간: 13초성공 리워드: 2원

5단계4x4 / 총 그림 8개- 기본 카드: 6개- 트릭 카드: 2개제한 시간: 15초성공 리워드: 3원

6단계4x5 / 총 그림 10개- 기본 카드: 10개- 트릭 카드: 0개제한 시간: 21초성공 리워드: 3원

7단계4x5 / 총 그림 10개- 기본 카드: 8개- 트릭 카드: 2개제한 시간: 25초성공 리워드: 6원

In [30]:
import random
import numpy as np
from collections import deque, Counter
from typing import List, Tuple, Dict
import pandas as pd

class Card:
    """카드 클래스"""
    def __init__(self, color: int, position: int):
        self.color = color
        self.position = position
        self.is_matched = False

class LimitedMemoryPlayer:
    """최근 N개 카드만 기억하는 플레이어 클래스"""
    def __init__(self, memory_size: int = 3):
        self.memory_size = memory_size
        self.memory = {}  # position -> color 매핑
        self.memory_order = deque(maxlen=memory_size)  # FIFO 순서 관리
        
    def remember_card(self, position: int, color: int):
        """카드를 기억에 추가"""
        if position in self.memory:
            self.memory_order.remove(position)
            self.memory_order.append(position)
        else:
            if len(self.memory) >= self.memory_size:
                oldest_pos = self.memory_order.popleft()
                del self.memory[oldest_pos]
            
            self.memory[position] = color
            self.memory_order.append(position)
    
    def find_matching_card(self, target_color: int, exclude_positions: List[int] = None) -> int:
        """기억에서 같은 색상의 카드 위치 찾기"""
        if exclude_positions is None:
            exclude_positions = []
            
        for position, color in self.memory.items():
            if color == target_color and position not in exclude_positions:
                return position
        return -1
    
    def remove_from_memory(self, positions: List[int]):
        """매칭된 카드들을 기억에서 제거"""
        for pos in positions:
            if pos in self.memory:
                del self.memory[pos]
                if pos in self.memory_order:
                    self.memory_order.remove(pos)

def create_deck(n_pairs: int) -> List[Card]:
    """n개 쌍의 카드 덱 생성"""
    deck = []
    colors = list(range(n_pairs)) * 2
    random.shuffle(colors)
    
    for i, color in enumerate(colors):
        deck.append(Card(color, i))
    
    return deck

def get_available_positions(deck: List[Card]) -> List[int]:
    """아직 매칭되지 않은 카드 위치들 반환"""
    return [i for i, card in enumerate(deck) if not card.is_matched]

def simulate_single_game(n_pairs: int, memory_size: int) -> Tuple[int, bool]:
    """단일 게임 시뮬레이션"""
    deck = create_deck(n_pairs)
    player = LimitedMemoryPlayer(memory_size)
    turns = 0
    matched_pairs = 0
    
    while matched_pairs < n_pairs:
        turns += 1
        
        # 사용 가능한 카드들
        available_positions = get_available_positions(deck)
        
        if len(available_positions) < 2:
            break
        
        # 첫 번째 카드 선택 (랜덤)
        first_pos = random.choice(available_positions)
        first_card = deck[first_pos]
        
        # 첫 번째 카드를 기억에 추가
        player.remember_card(first_pos, first_card.color)
        
        # 기억에서 같은 색상의 카드 찾기
        matching_pos = player.find_matching_card(first_card.color, [first_pos])
        
        if matching_pos != -1 and matching_pos in available_positions:
            # 기억에서 매칭 카드를 찾은 경우
            second_pos = matching_pos
            second_card = deck[second_pos]
            
            # 매칭 성공
            first_card.is_matched = True
            second_card.is_matched = True
            matched_pairs += 1
            
            # 매칭된 카드들을 기억에서 제거
            player.remove_from_memory([first_pos, second_pos])
        else:
            # 기억에 없으므로 랜덤하게 두 번째 카드 선택
            available_for_second = [pos for pos in available_positions if pos != first_pos]
            
            if not available_for_second:
                break
                
            second_pos = random.choice(available_for_second)
            second_card = deck[second_pos]
            
            # 두 번째 카드를 기억에 추가
            player.remember_card(second_pos, second_card.color)
            
            if first_card.color == second_card.color:
                # 운 좋게 매칭
                first_card.is_matched = True
                second_card.is_matched = True
                matched_pairs += 1
                
                # 매칭된 카드들을 기억에서 제거
                player.remove_from_memory([first_pos, second_pos])
    
    return turns, matched_pairs == n_pairs

def calculate_turn_probability_density(turns_list: List[int], max_turns: int) -> Dict[int, float]:
    """각 턴당 확률 밀도 계산"""
    turn_counter = Counter(turns_list)
    total_games = len(turns_list)
    
    probability_density = {}
    for turn in range(1, max_turns + 1):
        count = turn_counter.get(turn, 0)
        probability_density[f"turn_{turn}"] = count / total_games
    
    return probability_density

def run_detailed_simulation(n_pairs: int, memory_size: int, num_simulations: int = 10000) -> Dict:
    """상세 결과를 포함한 시뮬레이션 실행"""
    turns_results = []
    successful_games = 0
    
    # 시뮬레이션 실행
    for _ in range(num_simulations):
        turns, success = simulate_single_game(n_pairs, memory_size)
        turns_results.append(turns)
        if success:
            successful_games += 1
    
    # 기본 통계 계산
    min_turns = min(turns_results)
    max_turns = max(turns_results)
    average_turns = np.mean(turns_results)
    std_turns = np.std(turns_results)
    success_rate = successful_games / num_simulations
    
    # 확률 밀도 계산
    probability_density = calculate_turn_probability_density(turns_results, max_turns)
    
    # 결과 딕셔너리 구성
    result = {
        'n_pairs': n_pairs,
        'memory_size': memory_size,
        'num_simulations': num_simulations,
        'min_turns': min_turns,
        'max_turns': max_turns,
        'average_turns': round(average_turns, 3),
        'std_turns': round(std_turns, 3),
        'success_rate': round(success_rate, 4),
        **probability_density  # turn_1, turn_2, ... 확률 밀도 추가
    }
    
    return result

def run_batch_detailed_simulation(n_pairs_list: List[int], 
                                memory_sizes: List[int], 
                                num_simulations: int = 10000) -> List[Dict]:
    """여러 조건에 대한 배치 시뮬레이션"""
    results = []
    
    for n_pairs in n_pairs_list:
        for memory_size in memory_sizes:
            print(f"시뮬레이션 중: {n_pairs}쌍, 메모리 크기 {memory_size}...")
            result = run_detailed_simulation(n_pairs, memory_size, num_simulations)
            results.append(result)
    
    return results

def normalize_probability_densities(results: List[Dict]) -> List[Dict]:
    """확률 밀도를 가장 큰 max_turns에 맞춰 정규화"""
    # 모든 결과에서 최대 턴 수 찾기
    global_max_turns = max(result['max_turns'] for result in results)
    
    normalized_results = []
    for result in results:
        normalized_result = result.copy()
        
        # 부족한 턴에 대해 0.0으로 채우기
        for turn in range(1, global_max_turns + 1):
            turn_key = f"turn_{turn}"
            if turn_key not in normalized_result:
                normalized_result[turn_key] = 0.0
        
        normalized_results.append(normalized_result)
    
    return normalized_results

def results_to_dataframe(results: List[Dict]) -> pd.DataFrame:
    """결과를 DataFrame으로 변환"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # 열 순서 정렬 (기본 정보 + turn_1, turn_2, ...)
    basic_cols = ['n_pairs', 'memory_size', 'num_simulations', 'min_turns', 
                  'max_turns', 'average_turns', 'std_turns', 'success_rate']
    
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))  # turn_1, turn_2, ... 순서로 정렬
    
    column_order = basic_cols + turn_cols
    df = df[column_order]
    
    return df

def print_summary_statistics(results: List[Dict]):
    """요약 통계 출력"""
    print("\n=== 시뮬레이션 결과 요약 ===")
    print("=" * 80)
    
    for result in results:
        n_pairs = result['n_pairs']
        memory_size = result['memory_size']
        avg_turns = result['average_turns']
        std_turns = result['std_turns']
        min_turns = result['min_turns']
        max_turns = result['max_turns']
        success_rate = result['success_rate']
        
        print(f"\n【{n_pairs}쌍, 메모리 {memory_size}개】")
        print(f"  평균 턴 수: {avg_turns:.3f} ± {std_turns:.3f}")
        print(f"  범위: {min_turns} ~ {max_turns} 턴")
        print(f"  성공률: {success_rate:.1%}")
        
        # 확률이 높은 상위 3개 턴 출력
        turn_probs = [(k, v) for k, v in result.items() if k.startswith('turn_') and v > 0]
        turn_probs.sort(key=lambda x: x[1], reverse=True)
        
        print("  가장 빈번한 턴 수:")
        for turn_key, prob in turn_probs[:3]:
            turn_num = turn_key.split('_')[1]
            print(f"    {turn_num}턴: {prob:.1%}")

# 실행 예제
if __name__ == "__main__":
    print("=== 상세 결과 카드 게임 시뮬레이션 ===\n")
    
    # 시뮬레이션 설정
    n_pairs_list = [2, 3, 4]  # 카드 쌍 수
    memory_sizes = [1, 2, 3, 5]  # 메모리 크기
    num_simulations = 5000  # 시뮬레이션 횟수
    
    # 배치 시뮬레이션 실행
    results = run_batch_detailed_simulation(n_pairs_list, memory_sizes, num_simulations)
    
    # 요약 통계 출력
    print_summary_statistics(results)
    
    # DataFrame으로 변환
    df = results_to_dataframe(results)
    
    print(f"\n=== DataFrame 형태 결과 ===")
    print("형태:", df.shape)
    print("\n기본 정보 열:")
    basic_cols = ['n_pairs', 'memory_size', 'average_turns', 'min_turns', 'max_turns', 'success_rate']
    print(df[basic_cols].to_string(index=False))
    
    print(f"\n=== 확률 밀도 예시 (첫 번째 결과) ===")
    first_result = results[0]
    print(f"{first_result['n_pairs']}쌍, 메모리 {first_result['memory_size']}개:")
    
    turn_cols = [k for k in first_result.keys() if k.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    for turn_key in turn_cols[:10]:  # 처음 10턴만 출력
        turn_num = turn_key.split('_')[1]
        prob = first_result[turn_key]
        if prob > 0:
            print(f"  {turn_num}턴: {prob:.4f} ({prob:.1%})")
    
    # 이론적 완벽 플레이어와 비교
    print(f"\n=== 이론적 완벽 플레이어 비교 ===")
    for n_pairs in n_pairs_list:
        theoretical_perfect = 1.61 * n_pairs
        print(f"\n{n_pairs}쌍:")
        print(f"  이론적 완벽 플레이어: {theoretical_perfect:.2f}턴")
        
        # 해당 쌍에 대한 결과들 출력
        pair_results = [r for r in results if r['n_pairs'] == n_pairs]
        for result in pair_results:
            memory_size = result['memory_size']
            avg_turns = result['average_turns']
            diff = avg_turns - theoretical_perfect
            print(f"  메모리 {memory_size}개: {avg_turns:.2f}턴 (차이: {diff:+.2f})")
    
    # 전체 결과 반환을 위한 함수도 제공
    def get_simulation_results():
        """시뮬레이션 결과 반환 (다른 스크립트에서 사용 가능)"""
        return results, df, prob_df

=== 상세 결과 카드 게임 시뮬레이션 ===

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 4쌍, 메모리 크기 1...
시뮬레이션 중: 4쌍, 메모리 크기 2...
시뮬레이션 중: 4쌍, 메모리 크기 3...
시뮬레이션 중: 4쌍, 메모리 크기 5...

=== 시뮬레이션 결과 요약 ===

【2쌍, 메모리 1개】
  평균 턴 수: 4.015 ± 2.510
  범위: 2 ~ 24 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 33.5%
    3턴: 22.8%
    4턴: 13.8%

【2쌍, 메모리 2개】
  평균 턴 수: 3.345 ± 1.473
  범위: 2 ~ 12 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 33.6%
    3턴: 32.1%
    4턴: 16.7%

【2쌍, 메모리 3개】
  평균 턴 수: 2.987 ± 0.920
  범위: 2 ~ 9 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 43.9%
    2턴: 32.7%
    4턴: 17.2%

【2쌍, 메모리 5개】
  평균 턴 수: 2.945 ± 0.892
  범위: 2 ~ 8 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 43.5%
    2턴: 34.4%
    4턴: 16.8%

【3쌍, 메모리 1개】
  평균 턴 수: 9.020 ± 5.275
  범위: 3 ~ 45 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    6턴: 10.9%
    5턴: 10.6%
    4턴: 10.5%

【3쌍, 메모리 2개】
  평균 턴 수: 6.754 ± 2.902
 

In [31]:
# 시뮬레이션 실행
results = run_batch_detailed_simulation([2, 3, 4], [1, 2, 3, 5], 5000)

# 요청하신 형태의 DataFrame 추출
prob_df = extract_probability_density_dataframe(results)

print(prob_df.shape)  # (12, 20) 형태 (예시)
print(prob_df.index.names)  # ['n_pairs', 'memory_size']
print(prob_df.columns[:5])  # ['turn_1', 'turn_2', 'turn_3', 'turn_4', 'turn_5']

# 특정 조건 조회
print(prob_df.loc[(3, 2)])  # 3쌍, 메모리 2개 조건의 확률 분포
print(prob_df.loc[(3, 2), 'turn_5'])  # 5턴에 완료될 확률

# 슬라이싱
print(prob_df.loc[(2, 1):(3, 2), 'turn_1':'turn_10'])  # 부분 슬라이싱

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 4쌍, 메모리 크기 1...
시뮬레이션 중: 4쌍, 메모리 크기 2...
시뮬레이션 중: 4쌍, 메모리 크기 3...
시뮬레이션 중: 4쌍, 메모리 크기 5...


NameError: name 'extract_probability_density_dataframe' is not defined

In [27]:
result, df= get_simulation_results()

result[10]

{'n_pairs': 3,
 'memory_size': 3,
 'num_simulations': 5000,
 'min_turns': 3,
 'max_turns': 15,
 'average_turns': 5.385,
 'std_turns': 1.662,
 'success_rate': 1.0,
 'turn_1': 0.0,
 'turn_2': 0.0,
 'turn_3': 0.0632,
 'turn_4': 0.2722,
 'turn_5': 0.2842,
 'turn_6': 0.1834,
 'turn_7': 0.0934,
 'turn_8': 0.0528,
 'turn_9': 0.0224,
 'turn_10': 0.0152,
 'turn_11': 0.0068,
 'turn_12': 0.004,
 'turn_13': 0.0016,
 'turn_14': 0.0004,
 'turn_15': 0.0004}